In [2]:
import pandas as pd
df = pd.read_csv('/content/IMDB Dataset.csv')
df.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   review     50000 non-null  object
 1   sentiment  50000 non-null  object
dtypes: object(2)
memory usage: 781.4+ KB


In [4]:
df.isnull().sum()

,0
review,0
sentiment,0


In [5]:
df.duplicated().sum()

418

In [6]:
df.drop_duplicates()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive
...,...,...
49995,I thought this movie did a down right good job...,positive
49996,"Bad plot, bad dialogue, bad acting, idiotic di...",negative
49997,I am a Catholic taught in parochial elementary...,negative
49998,I'm going to have to disagree with the previou...,negative


In [7]:
df['sentiment'].unique()

array(['positive', 'negative'], dtype=object)

In [8]:
df['sentiment'].value_counts()

,count
sentiment,
positive,25000
negative,25000


In [9]:
Sentiment_classes = {'negative': 0, 'positive': 1 }
df['sentiment'] = df['sentiment'].map(Sentiment_classes)
df

,review,sentiment
0,One of the other reviewers has mentioned that ...,1
1,A wonderful little production. <br /><br />The...,1
2,I thought this was a wonderful way to spend ti...,1
3,Basically there's a family where a little boy ...,0
4,"Petter Mattei's ""Love in the Time of Money"" is...",1
...,...,...
49995,I thought this movie did a down right good job...,1
49996,"Bad plot, bad dialogue, bad acting, idiotic di...",0
49997,I am a Catholic taught in parochial elementary...,0
49998,I'm going to have to disagree with the previou...,0


In [10]:
import string
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer, PorterStemmer
from nltk.corpus import wordnet
from nltk import pos_tag
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, classification_report

In [11]:
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


True

In [12]:
import re
def clean_text(text):
    # Remove words starting with '@'
    text = re.sub(r'@\w+', '', text)
    # Remove words starting with 'https://'
    text = re.sub(r'https://\S+', '', text)
    # Remove extra spaces
    text = re.sub(r'\s+', ' ', text).strip()
    # Ensure the text ends with a period
    text = re.sub(r'#','',text)
    if not text.endswith('.'):
        text += '.'
    return text
df['Clean_review'] = df['review'].apply(clean_text)
df

,review,sentiment,Clean_review
0,One of the other reviewers has mentioned that ...,1,One of the other reviewers has mentioned that ...
1,A wonderful little production. <br /><br />The...,1,A wonderful little production. <br /><br />The...
2,I thought this was a wonderful way to spend ti...,1,I thought this was a wonderful way to spend ti...
3,Basically there's a family where a little boy ...,0,Basically there's a family where a little boy ...
4,"Petter Mattei's ""Love in the Time of Money"" is...",1,"Petter Mattei's ""Love in the Time of Money"" is..."
...,...,...,...
49995,I thought this movie did a down right good job...,1,I thought this movie did a down right good job...
49996,"Bad plot, bad dialogue, bad acting, idiotic di...",0,"Bad plot, bad dialogue, bad acting, idiotic di..."
49997,I am a Catholic taught in parochial elementary...,0,I am a Catholic taught in parochial elementary...
49998,I'm going to have to disagree with the previou...,0,I'm going to have to disagree with the previou...


In [13]:
import string
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
def preprocess_text(text):
    # Convert the text to lowercase
    text = text.lower()
    # Clean the text
    text = clean_text(text)
    # Tokenize the text
    tokens = word_tokenize(text)
    # Remove punctuation
    tokens = [word for word in tokens if word.isalnum()]  # Only keep alphanumeric tokens
    # Remove stopwords
    stop_words = set(stopwords.words('english'))
    tokens = [word for word in tokens if word not in stop_words]
    # Initialize lemmatizer
    lemmatizer = WordNetLemmatizer()
    # Lemmatize the tokens
    tokens = [lemmatizer.lemmatize(word) for word in tokens]
    # Reconstruct the text from tokens
    text = ' '.join(tokens)
    return text, tokens
df['preprocess_text']=df['Clean_review'].apply(preprocess_text)
df

,review,sentiment,Clean_review,preprocess_text
0,One of the other reviewers has mentioned that ...,1,One of the other reviewers has mentioned that ...,(one reviewer mentioned watching 1 oz episode ...
1,A wonderful little production. <br /><br />The...,1,A wonderful little production. <br /><br />The...,(wonderful little production br br filming tec...
2,I thought this was a wonderful way to spend ti...,1,I thought this was a wonderful way to spend ti...,(thought wonderful way spend time hot summer w...
3,Basically there's a family where a little boy ...,0,Basically there's a family where a little boy ...,(basically family little boy jake think zombie...
4,"Petter Mattei's ""Love in the Time of Money"" is...",1,"Petter Mattei's ""Love in the Time of Money"" is...",(petter mattei love time money visually stunni...
...,...,...,...,...
49995,I thought this movie did a down right good job...,1,I thought this movie did a down right good job...,(thought movie right good job creative origina...
49996,"Bad plot, bad dialogue, bad acting, idiotic di...",0,"Bad plot, bad dialogue, bad acting, idiotic di...",(bad plot bad dialogue bad acting idiotic dire...
49997,I am a Catholic taught in parochial elementary...,0,I am a Catholic taught in parochial elementary...,(catholic taught parochial elementary school n...
49998,I'm going to have to disagree with the previou...,0,I'm going to have to disagree with the previou...,(going disagree previous comment side maltin o...


In [14]:
df['tokens'] = df['preprocess_text'].apply(lambda x: x[1])
df

,review,sentiment,Clean_review,preprocess_text,tokens
0,One of the other reviewers has mentioned that ...,1,One of the other reviewers has mentioned that ...,(one reviewer mentioned watching 1 oz episode ...,"[one, reviewer, mentioned, watching, 1, oz, ep..."
1,A wonderful little production. <br /><br />The...,1,A wonderful little production. <br /><br />The...,(wonderful little production br br filming tec...,"[wonderful, little, production, br, br, filmin..."
2,I thought this was a wonderful way to spend ti...,1,I thought this was a wonderful way to spend ti...,(thought wonderful way spend time hot summer w...,"[thought, wonderful, way, spend, time, hot, su..."
3,Basically there's a family where a little boy ...,0,Basically there's a family where a little boy ...,(basically family little boy jake think zombie...,"[basically, family, little, boy, jake, think, ..."
4,"Petter Mattei's ""Love in the Time of Money"" is...",1,"Petter Mattei's ""Love in the Time of Money"" is...",(petter mattei love time money visually stunni...,"[petter, mattei, love, time, money, visually, ..."
...,...,...,...,...,...
49995,I thought this movie did a down right good job...,1,I thought this movie did a down right good job...,(thought movie right good job creative origina...,"[thought, movie, right, good, job, creative, o..."
49996,"Bad plot, bad dialogue, bad acting, idiotic di...",0,"Bad plot, bad dialogue, bad acting, idiotic di...",(bad plot bad dialogue bad acting idiotic dire...,"[bad, plot, bad, dialogue, bad, acting, idioti..."
49997,I am a Catholic taught in parochial elementary...,0,I am a Catholic taught in parochial elementary...,(catholic taught parochial elementary school n...,"[catholic, taught, parochial, elementary, scho..."
49998,I'm going to have to disagree with the previou...,0,I'm going to have to disagree with the previou...,(going disagree previous comment side maltin o...,"[going, disagree, previous, comment, side, mal..."


In [15]:
from sklearn.feature_extraction.text import TfidfVectorizer

# CountVectorizer object for unigrams and bigrams
vectorizer = TfidfVectorizer()

# Extract the preprocessed text
preprocessed_texts = df['preprocess_text'].apply(lambda x: x[0])

# Fit and transform the preprocessed text data
X = vectorizer.fit_transform(preprocessed_texts)
y = df['sentiment']

print(X.shape)

(50000, 89104)


In [16]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size= 0.2, random_state= 40)
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

(40000, 89104) (10000, 89104) (40000,) (10000,)


In [17]:
NB_clf = MultinomialNB()
NB_clf.fit(X_train, y_train)

MultinomialNB()

In [18]:
NB_pred = NB_clf.predict(X_test)

In [19]:
print('NB Accuracy', accuracy_score(y_test, NB_pred))
print("NB Classification Report:\n", classification_report(y_test, NB_pred))

NB Accuracy 0.8611
NB Classification Report:
               precision    recall  f1-score   support

           0       0.86      0.87      0.86      5100
           1       0.86      0.85      0.86      4900

    accuracy                           0.86     10000
   macro avg       0.86      0.86      0.86     10000
weighted avg       0.86      0.86      0.86     10000



In [20]:
# Decision Tree classifier
DT_clf = DecisionTreeClassifier()
DT_clf.fit(X_train, y_train)

DecisionTreeClassifier()

In [21]:
# Predict
DT_pred = DT_clf.predict(X_test)

In [22]:
# Evaluate
print("Decision Tree Accuracy:", accuracy_score(y_test, DT_pred))
print("Decision Tree Classification Report:\n", classification_report(y_test, DT_pred))

Decision Tree Accuracy: 0.7232
Decision Tree Classification Report:
               precision    recall  f1-score   support

           0       0.73      0.73      0.73      5100
           1       0.72      0.72      0.72      4900

    accuracy                           0.72     10000
   macro avg       0.72      0.72      0.72     10000
weighted avg       0.72      0.72      0.72     10000



In [23]:
# Hp T Naive Bayes
from sklearn.model_selection import GridSearchCV

In [24]:
# the parameter grid for Naive Bayes
param_grid_nb = {
    'alpha': [5.0, 6.0, 7.0],
    'fit_prior': [True, False],
    'class_prior': [None,[0.4, 0.6]]
}

nb_model = MultinomialNB()

In [25]:
# GridSearchCV
grid_search_nb = GridSearchCV(estimator=nb_model, param_grid=param_grid_nb, cv=5, scoring='accuracy')

# Fit GridSearchCV
grid_search_nb.fit(X_train, y_train)

GridSearchCV(cv=5, estimator=MultinomialNB(),
             param_grid={'alpha': [5.0, 6.0, 7.0],
                         'class_prior': [None, [0.4, 0.6]],
                         'fit_prior': [True, False]},
             scoring='accuracy')

In [26]:
print("Best Parameters for Naive Bayes:", grid_search_nb.best_params_)
print("Best Score for Naive Bayes:", grid_search_nb.best_score_)

Best Parameters for Naive Bayes: {'alpha': 5.0, 'class_prior': None, 'fit_prior': True}
Best Score for Naive Bayes: 0.86445


In [27]:
# the parameter grid for Decision Tree
param_grid_dt = {
    'max_depth': [None, 10, 20],
    'min_samples_split': [2, 5],
    'min_samples_leaf': [1, 2],
    'criterion': ['gini']
}
dt_model = DecisionTreeClassifier()

In [28]:
# GridSearchCV
grid_search_dt = GridSearchCV(estimator=dt_model, param_grid=param_grid_dt, cv=4, verbose=2, scoring='accuracy')

# Fit GridSearchCV
grid_search_dt.fit(X_train, y_train)

Fitting 4 folds for each of 12 candidates, totalling 48 fits
[CV] END criterion=gini, max_depth=None, min_samples_leaf=1, min_samples_split=2; total time= 1.2min
[CV] END criterion=gini, max_depth=None, min_samples_leaf=1, min_samples_split=2; total time= 1.2min
[CV] END criterion=gini, max_depth=None, min_samples_leaf=1, min_samples_split=2; total time= 1.3min
[CV] END criterion=gini, max_depth=None, min_samples_leaf=1, min_samples_split=2; total time= 1.2min
[CV] END criterion=gini, max_depth=None, min_samples_leaf=1, min_samples_split=5; total time= 1.2min
[CV] END criterion=gini, max_depth=None, min_samples_leaf=1, min_samples_split=5; total time= 1.2min
[CV] END criterion=gini, max_depth=None, min_samples_leaf=1, min_samples_split=5; total time= 1.2min
[CV] END criterion=gini, max_depth=None, min_samples_leaf=1, min_samples_split=5; total time= 1.1min
[CV] END criterion=gini, max_depth=None, min_samples_leaf=2, min_samples_split=2; total time= 1.1min
[CV] END criterion=gini, max_d

GridSearchCV(cv=4, estimator=DecisionTreeClassifier(),
             param_grid={'criterion': ['gini'], 'max_depth': [None, 10, 20],
                         'min_samples_leaf': [1, 2],
                         'min_samples_split': [2, 5]},
             scoring='accuracy', verbose=2)

In [29]:
print("Best Parameters for Decision Tree:", grid_search_dt.best_params_)
print("Best Score for Decision Tree:", grid_search_dt.best_score_)

Best Parameters for Decision Tree: {'criterion': 'gini', 'max_depth': 20, 'min_samples_leaf': 1, 'min_samples_split': 5}
Best Score for Decision Tree: 0.7351
